In [1]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm


import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision import models
import torch.optim as optim
from torch.utils.data import DataLoader
class AgeDataset(torch.utils.data.Dataset):

    def __init__(self,  data_path, annot_path, train=True):
        super(AgeDataset, self).__init__()

        self.annot_path = annot_path
        self.data_path = data_path
        self.train = train
        
        self.ann = pd.read_csv(annot_path)
        self.files = self.ann['file_id']
        if train:
            self.ages = self.ann['age']
        self.transform = self._transform(224)

    @staticmethod    
    def _convert_image_to_rgb(image):
        return image.convert("RGB")

    def _transform(self, n_px):
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        return Compose([
            Resize(n_px),
            self._convert_image_to_rgb,
            ToTensor(),
            Normalize(mean, std),
        ])

    def read_img(self, file_name):
        im_path = join(self.data_path,file_name)   
        img = Image.open(im_path)
        img = self.transform(img)
        return img

    def __getitem__(self, index):
        file_name = self.files[index]
        img = self.read_img(file_name)
        if self.train:
            age = self.ages[index]
            return img, age
        else:
            return img

    def __len__(self):
        return len(self.files)


train_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
train_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'
dataset = AgeDataset(train_path, train_ann, train=True)
train_dataset,val_dataset = torch.utils.data.random_split(dataset,[0.8,0.2])


test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'
test_dataset = AgeDataset(test_path, test_ann, train=False)

In [2]:
len(train_dataset),len(val_dataset),len(test_dataset)

(17072, 4268, 1950)

In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=64,shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


@torch.no_grad
def predict(loader, model):
    model.eval()
    predictions = []

    for img in tqdm(loader):
        img = img.to(device)

        pred = model(img)
        predictions.extend(pred.flatten().detach().tolist())

    return predictions

def validation_loop(val_loader,model,criterion):
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for images,ages in tqdm(val_loader):
            images = images.to(device)
            ages = ages.to(device).float().unsqueeze(1)  # Ensure age is a float and has correct shape
            outputs = model(images)
            loss = criterion(outputs, ages)
            running_val_loss += loss.item() * images.size(0)
            
    print(f"Val Loss: {running_val_loss/len(val_loader.dataset)}")
        
    

# Training Loop
def train_model(model, train_loader,val_loader,test_loader, criterion, optimizer, num_epochs=5):
      # Set the model to training mode
    for epoch in range(num_epochs):
        train_running_loss = 0.0
        model.train()
        for images, ages in tqdm(train_loader):
            images = images.to(device)
            ages = ages.to(device).float().unsqueeze(1)  # Ensure age is a float and has correct shape

            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(images)
            loss = criterion(outputs, ages)
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Optimize the weights

            train_running_loss += loss.item() * images.size(0)
        
        epoch_loss = train_running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}, Traing Loss: {epoch_loss:.4f}, ",end="")
        
        validation_loop(val_loader,model,criterion)
        
        preds = predict(test_loader, model)
        submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
        submit['age'] = preds
        submit.to_csv(f'/kaggle/working/submission_epoch_{epoch+1}.csv',index=False)            
        

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)  # Example using a pre-trained ResNet50

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
                        nn.Linear(num_ftrs, 1)
                        )# Assuming age prediction is a regression task
for name,param in model.named_parameters():
    if "layer" in name:
        if "layer1" in name:
            param.requires_grad = False
    elif 'fc.' not in name:
        param.requires_grad = False
model = model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 93.7MB/s]


In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
for name,param in model.named_parameters():
    print(f"{name} requires Grad: {param.requires_grad}")

conv1.weight requires Grad: False
bn1.weight requires Grad: False
bn1.bias requires Grad: False
layer1.0.conv1.weight requires Grad: False
layer1.0.bn1.weight requires Grad: False
layer1.0.bn1.bias requires Grad: False
layer1.0.conv2.weight requires Grad: False
layer1.0.bn2.weight requires Grad: False
layer1.0.bn2.bias requires Grad: False
layer1.1.conv1.weight requires Grad: False
layer1.1.bn1.weight requires Grad: False
layer1.1.bn1.bias requires Grad: False
layer1.1.conv2.weight requires Grad: False
layer1.1.bn2.weight requires Grad: False
layer1.1.bn2.bias requires Grad: False
layer2.0.conv1.weight requires Grad: True
layer2.0.bn1.weight requires Grad: True
layer2.0.bn1.bias requires Grad: True
layer2.0.conv2.weight requires Grad: True
layer2.0.bn2.weight requires Grad: True
layer2.0.bn2.bias requires Grad: True
layer2.0.downsample.0.weight requires Grad: True
layer2.0.downsample.1.weight requires Grad: True
layer2.0.downsample.1.bias requires Grad: True
layer2.1.conv1.weight requi

In [6]:
# Setup loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Call the train_model function
train_model(model, train_loader,val_loader,test_loader, criterion, optimizer, num_epochs=30)


100%|██████████| 267/267 [03:08<00:00,  1.42it/s]


Epoch 1, Traing Loss: 668.6255, 

100%|██████████| 67/67 [00:44<00:00,  1.51it/s]


Val Loss: 352.2997142788173


100%|██████████| 267/267 [01:09<00:00,  3.85it/s]


...

100%|██████████| 267/267 [01:07<00:00,  3.96it/s]


Epoch 30, Traing Loss: 2.7443, 

100%|██████████| 67/67 [00:14<00:00,  4.50it/s]


Val Loss: 47.143300561449074


100%|██████████| 31/31 [00:06<00:00,  4.65it/s]


In [7]:
###### SUBMISSION CSV FILE #####
preds = predict(test_loader, model)

submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
submit['age'] = preds
submit.head()

submit.to_csv('/kaggle/working/submission.csv',index=False)

100%|██████████| 31/31 [00:06<00:00,  4.54it/s]
